## reading and get into proper format training data

In [1]:
import csv
import scipy.io
import numpy as np
data = scipy.io.loadmat("average_abs_slope_no_pca.mat")

for i in data:
    if '__' not in i and 'readme' not in i:
          np.savetxt(("average_abs_slope_no_pca.csv"),data[i],delimiter=',')

# every 30 lines is a different speaker. Each recording is decomposed into 9 arrays of 4 points (9 lines from the original plot)
with open('average_abs_slope_no_pca.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))
#print(len(data))
# transform data into arrays of 5 times 12
temp=[]
c=[]
i=0
for i in range(0,1620,6):
    #print(len(elem))
    for t in range(6):
        #print("value i=",i)
        #print("value t=",t)
        c+= data[i+t]

    temp.append(c)
    c=[]
#print("packs of 72: ",temp[1])

f=[]
final=[]
for elem in temp:
    for i in range(0,72,6):
        f.append(elem[0+i:6+i])
    final.append(f)
    f=[]
#print("herw:",final[0])
x_train=final

## reading and get into proper shape testing data

In [2]:
import csv
import scipy.io
import numpy as np
test = scipy.io.loadmat("average_abs_slope_no_pca_test_data.mat")

for i in test:
    if '__' not in i and 'readme' not in i:
          np.savetxt(("average_abs_slope_no_pca_test_data.csv"),test[i],delimiter=',')
            
# every 30 lines is a different speaker. Each recording is decomposed into 9 arrays of 4 points (9 lines from the original plot)
with open('average_abs_slope_no_pca_test_data.csv', newline='') as csvfile:
    test = list(csv.reader(csvfile))

temp=[]
c=[]
i=0
for i in range(0,2220,6):
    #print(len(elem))
    for t in range(6):
        #print("value i=",i)
        #print("value t=",t)
        c+= test[i+t]

    temp.append(c)
    c=[]
#print("packs of 72: ",temp[0])

f=[]
final=[]
for elem in temp:
    for i in range(0,72,6):
        f.append(elem[0+i:6+i])
    final.append(f)
    f=[]
x_test=final

## create labels and do one-hot encoding

In [3]:
integer_encoded = []
for i in range(9):
    integer_encoded += [i] * 30

onehot_encoded = []
for elem in integer_encoded:
    letter = [0 for i in range(9)]
    letter[elem] = 1
    onehot_encoded.append(letter)

y_train=onehot_encoded


# generate labels for the training 
integer_encoded = [0]*31 + [1]*35 + [2]*88 + [3]*44 + [4]*29 + [5]*24 + [6]*40 + [7]*50 + [8]*29

onehot_encoded = []
for elem in integer_encoded:
    letter = [0 for i in range(9)]
    letter[elem] = 1
    onehot_encoded.append(letter)

y_test=onehot_encoded


## Put strings into floats for both training and testing data

In [4]:
import string

def Convert_to_list_of_ints(data): 
    t=[]
    c=[]
    # there are 270 elements
    for elem in data:
        for i in range(12):
            temp=[float(x) for x in elem[i]]
            c.append(temp) #concatenate the sub arrays for each of the 9 lines to make one big array
        t.append(c)
        c=[]
    return t

x_train=Convert_to_list_of_ints(x_train)
x_test=Convert_to_list_of_ints(x_test)


## shuffling training and testing data

In [5]:
import numpy as np
import random

# shuffle training data
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

## Turn the lists into np.arrays
x_train, y_train = np.array(x_train), np.array(y_train)

## we also have to convert our data into three-dimensional format
x_train = np.reshape(x_train, (x_train.shape[0], 6,12))

# shuffle testing data
b = list(zip(x_test, y_test))
random.shuffle(b)
x_test, y_test = zip(*b)

## Turn the lists into np.arrays
x_test, y_test = np.array(x_test), np.array(y_test)

## we also have to convert our data into three-dimensional format
x_test = np.reshape(x_test, (x_test.shape[0], 6,12))

In [6]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding, BatchNormalization, Activation
from sklearn.model_selection import KFold
from keras.regularizers import L1L2

def create_model():
    model = Sequential()
##model.add(Bidirectional(LSTM(50, activation='relu',return_sequences=True,kernel_regularizer=L1L2(l1=0.0, l2=0.0)), input_shape=(x_train.shape[1], 9)))
    model.add(LSTM(units=90,activation='tanh', recurrent_activation='sigmoid', return_sequences=False,input_shape=(x_train.shape[1], 12),dropout=0.2,recurrent_dropout=0.2)) 
##model.add(BatchNormalization())
##model.add(LSTM(units=90,activation='relu', recurrent_activation='sigmoid', return_sequences=True,dropout=0.34,recurrent_dropout=0.34))
##model.add(LSTM(units=90, activation='relu', recurrent_activation='sigmoid',return_sequences=False,dropout=0.34,recurrent_dropout=0.34))
    model.add(Dense(units=9))
    model.add(Activation("linear")) 
    opt= keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer = opt, loss = 'mean_squared_error',metrics = ['accuracy'])
    return model
#model.fit(x_train, y_train, batch_size=27, epochs=2) #was 150

# saving the model
#model.save('keras_ML1.h5')


#FIRST TEST K-FOLD
def train_and_evaluate_model(model, data, labels, Data, Labels):
    model.fit(data, labels, batch_size=9, epochs=30) #was 150 #training subset
    score = model.evaluate(Data, Labels, verbose=0) #testing subset
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    return score
    # fit and evaluate here.

Test_Loss = []
Test_Accuracy = []
    
from sklearn.model_selection import KFold
#if __name__ == "__main__":
data, labels = x_train, y_train
kf = KFold(n_splits=10) 
i=0
for train, test in kf.split(x_train):
    print("%s %s" % (train, test))
    i=i+1
    print("Running Fold", i,"/", 10)
    model = None # Clearing the NN.
    model = create_model()
    score = train_and_evaluate_model(model, data[train], labels[train], data[test], labels[test])
    Test_Loss.append(score[0])
    Test_Accuracy.append(score[1])
    
model.save('keras_ML3.h5')

Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

[ 27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44
  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62
  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80
  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98
  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116
 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134
 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152
 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260
 261 262 263 264 265 266 267 268 269] [ 0  1  2  3 

243/243 [==============================] - 0s 525us/step - loss: 0.0364 - accuracy: 0.8765
Epoch 28/30
243/243 [==============================] - 0s 408us/step - loss: 0.0354 - accuracy: 0.8930
Epoch 29/30
243/243 [==============================] - 0s 420us/step - loss: 0.0386 - accuracy: 0.8519
Epoch 30/30
243/243 [==============================] - 0s 420us/step - loss: 0.0363 - accuracy: 0.8765
Test loss: 0.027399301528930664
Test accuracy: 1.0
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98
  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116
 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134
 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152
 153 154 155 156 157 158 159 160 161 1

Epoch 24/30
243/243 [==============================] - 0s 410us/step - loss: 0.0413 - accuracy: 0.8519
Epoch 25/30
243/243 [==============================] - 0s 415us/step - loss: 0.0438 - accuracy: 0.8272
Epoch 26/30
243/243 [==============================] - 0s 422us/step - loss: 0.0396 - accuracy: 0.8642
Epoch 27/30
243/243 [==============================] - 0s 421us/step - loss: 0.0373 - accuracy: 0.8683
Epoch 28/30
243/243 [==============================] - 0s 418us/step - loss: 0.0392 - accuracy: 0.8765
Epoch 29/30
243/243 [==============================] - 0s 414us/step - loss: 0.0396 - accuracy: 0.8477
Epoch 30/30
243/243 [==============================] - 0s 409us/step - loss: 0.0381 - accuracy: 0.8395
Test loss: 0.029530059546232224
Test accuracy: 1.0
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55 

243/243 [==============================] - 0s 412us/step - loss: 0.0441 - accuracy: 0.8436
Epoch 21/30
243/243 [==============================] - 0s 408us/step - loss: 0.0425 - accuracy: 0.8354
Epoch 22/30
243/243 [==============================] - 0s 428us/step - loss: 0.0418 - accuracy: 0.8519
Epoch 23/30
243/243 [==============================] - 0s 459us/step - loss: 0.0384 - accuracy: 0.8765
Epoch 24/30
243/243 [==============================] - 0s 468us/step - loss: 0.0417 - accuracy: 0.8189
Epoch 25/30
243/243 [==============================] - 0s 422us/step - loss: 0.0391 - accuracy: 0.8477
Epoch 26/30
243/243 [==============================] - 0s 436us/step - loss: 0.0404 - accuracy: 0.8642
Epoch 27/30
243/243 [==============================] - 0s 427us/step - loss: 0.0384 - accuracy: 0.8477
Epoch 28/30
243/243 [==============================] - 0s 561us/step - loss: 0.0381 - accuracy: 0.8519
Epoch 29/30
243/243 [==============================] - 0s 598us/step - loss: 0.0389 -

243/243 [==============================] - 0s 463us/step - loss: 0.0468 - accuracy: 0.7984
Epoch 18/30
243/243 [==============================] - 0s 496us/step - loss: 0.0458 - accuracy: 0.8189
Epoch 19/30
243/243 [==============================] - 0s 433us/step - loss: 0.0449 - accuracy: 0.8395
Epoch 20/30
243/243 [==============================] - 0s 439us/step - loss: 0.0435 - accuracy: 0.8436
Epoch 21/30
243/243 [==============================] - 0s 423us/step - loss: 0.0436 - accuracy: 0.8601
Epoch 22/30
243/243 [==============================] - 0s 445us/step - loss: 0.0403 - accuracy: 0.8189
Epoch 23/30
243/243 [==============================] - 0s 446us/step - loss: 0.0406 - accuracy: 0.8313
Epoch 24/30
243/243 [==============================] - 0s 451us/step - loss: 0.0385 - accuracy: 0.8642
Epoch 25/30
243/243 [==============================] - 0s 560us/step - loss: 0.0398 - accuracy: 0.8313
Epoch 26/30
243/243 [==============================] - 0s 473us/step - loss: 0.0393 -

243/243 [==============================] - 0s 531us/step - loss: 0.0493 - accuracy: 0.7984
Epoch 15/30
243/243 [==============================] - 0s 506us/step - loss: 0.0496 - accuracy: 0.7984
Epoch 16/30
243/243 [==============================] - 0s 507us/step - loss: 0.0478 - accuracy: 0.8230
Epoch 17/30
243/243 [==============================] - 0s 482us/step - loss: 0.0458 - accuracy: 0.8066
Epoch 18/30
243/243 [==============================] - 0s 469us/step - loss: 0.0423 - accuracy: 0.8477
Epoch 19/30
243/243 [==============================] - 0s 456us/step - loss: 0.0443 - accuracy: 0.8272
Epoch 20/30
243/243 [==============================] - 0s 476us/step - loss: 0.0415 - accuracy: 0.8642
Epoch 21/30
243/243 [==============================] - 0s 495us/step - loss: 0.0428 - accuracy: 0.8436
Epoch 22/30
243/243 [==============================] - 0s 521us/step - loss: 0.0414 - accuracy: 0.8519
Epoch 23/30
243/243 [==============================] - 0s 573us/step - loss: 0.0399 -

In [7]:
Test_Accuracy

[1.0,
 1.0,
 0.8148148059844971,
 1.0,
 0.9629629850387573,
 1.0,
 0.8148148059844971,
 1.0,
 0.9629629850387573,
 0.9259259104728699]

In [8]:
from keras.models import load_model

# testing the model
model = load_model('keras_ML3.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.02474042879971298
Test accuracy: 0.9594594836235046


In [9]:
from sklearn.metrics import classification_report
import numpy as np

Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(x_test)
print(classification_report(Y_test, y_pred))

             precision    recall  f1-score   support

          0       1.00      0.94      0.97        31
          1       0.85      0.94      0.89        35
          2       1.00      0.94      0.97        88
          3       1.00      0.95      0.98        44
          4       0.88      1.00      0.94        29
          5       0.96      1.00      0.98        24
          6       1.00      0.97      0.99        40
          7       0.92      0.98      0.95        50
          8       1.00      0.93      0.96        29

avg / total       0.96      0.96      0.96       370

